In [0]:

%pip install prophet pandas
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:

from pyspark.sql.functions import col
import pandas as pd
from prophet import Prophet

CATALOG = "workspace"
SCHEMA  = "default"
gold_tbl     = f"{CATALOG}.{SCHEMA}.stocks_gold_features"
forecast_tbl = f"{CATALOG}.{SCHEMA}.stocks_forecast"

symbol = "AAPL"  # you can loop over more later

# Load time series
df = (spark.table(gold_tbl)
          .filter(col("symbol") == symbol)
          .select("date", "close")
          .orderBy("date"))

pdf = df.toPandas().dropna()
pdf = pdf.rename(columns={"date":"ds","close":"y"})

# Train & forecast next 7 days
m = Prophet(daily_seasonality=True)
m.fit(pdf)
future   = m.make_future_dataframe(periods=7, freq="D")
forecast = m.predict(future)

preds = forecast[["ds","yhat","yhat_lower","yhat_upper"]].copy()
preds["symbol"] = symbol

# Overwrite managed forecast table (simple demo approach)
spark.createDataFrame(preds).write.mode("overwrite").saveAsTable(forecast_tbl)

display(spark.table(forecast_tbl).orderBy("ds"))


07:11:18 - cmdstanpy - INFO - Chain [1] start processing
07:11:18 - cmdstanpy - INFO - Chain [1] done processing


ds,yhat,yhat_lower,yhat_upper,symbol
2014-01-02T00:00:00.000Z,74.88599953750439,69.82659152295965,80.2446447548575,AAPL
2014-01-03T00:00:00.000Z,74.66122203711517,69.17465973087008,80.01321486466195,AAPL
2014-01-06T00:00:00.000Z,74.22171851901189,68.76954382114748,79.7134804679559,AAPL
2014-01-07T00:00:00.000Z,74.02679048295695,68.96239993703405,79.61990615090832,AAPL
2014-01-08T00:00:00.000Z,73.85109650404594,68.47018454391572,79.21592754677685,AAPL
2014-01-09T00:00:00.000Z,73.49422006773415,68.06412195527625,79.03690389325376,AAPL
2014-01-10T00:00:00.000Z,73.27663779637729,68.04043379024043,78.31947170586818,AAPL
2014-01-13T00:00:00.000Z,72.89185323387903,67.22874488680083,78.42912664223522,AAPL
2014-01-14T00:00:00.000Z,72.7317812774721,67.5081432994559,77.8614682390065,AAPL
2014-01-15T00:00:00.000Z,72.60187030117368,66.97757354211159,78.0829572866164,AAPL
